# 1. Encoding and Decoding the Image using LSB Steganography

### Steganography

Steganography is a technique used to hide secret information in a way that no one notices it’s even there. For this project, I am using an image to hide a message. This works because images are made of tiny dots called pixels, and each pixel has some color data (red, green, and blue). We can tweak this data slightly to store our message without making any visible changes to the image.

### LSB Steganography

LSB stands for **“Least Significant Bit.”** It is the smallest part of the color information for a pixel. By changing this tiny bit, we can hide our message. For example, if the red part of a pixel is `11001100`, changing the last bit (the “least significant” one) to `1` makes it `11001101`. The change is so small that the human eye cannot notice it. 

[Click me to learn 📚](https://www.researchgate.net/publication/369829880_Hiding_Information_in_Digital_Images_Using_LSB_Steganography_Technique)


## 1.1. Encode the Message in the Image

### Steps for Securing and Hiding the Message

#### Securing with a Passcode

I hashed a passcode using **SHA-256** to create a secure key. SHA-256 is a cryptographic hash function that generates a 256-bit (32-byte) fixed-length output. This ensures that only someone with the correct passcode can decode the message, as the hash adds a layer of security.

SHA-256 is widely used in secure applications due to its strong resistance to attacks and high computational efficiency. For more details on its performance and applications, see:  
[A High-Performance Multimem SHA-256 Accelerator for Society 5.0](https://www.researchgate.net/publication/349744176_A_High-Performance_Multimem_SHA-256_Accelerator_for_Society_50)


#### 2. Preparing the Message
The hashed passcode is added to the start of the secret message. I also add a marker (like the word `"Ending"`) to signal the end of the message.

#### 3. Converting to Binary
The entire message is converted into binary (a long string of 0s and 1s).

#### 4. Hiding the Message
Each bit of the binary message is stored in the **Least Significant Bit (LSB)** of the pixel colors (red, green, and blue). This process is done pixel by pixel, starting from the top-left corner of the image.


In [ ]:
import hashlib
from PIL import Image

def encode_message_lsb_with_passcode(image_path, secret_text, output_path, passcode):
    # Generate a hashed passcode using SHA-256
    hashed_passcode = hashlib.sha256(passcode.encode('utf-8')).hexdigest()
    
    # Append the hashed passcode to the secret text
    secret_text = hashed_passcode + secret_text + "Ending"
    
    # Open the image
    image = Image.open(image_path).convert("RGB")
    pixels = image.load()
    
    # Convert the secret text to binary (UTF-8 encoding)
    binary_secret_text = ''.join(format(byte, '08b') for byte in secret_text.encode('utf-8'))
    
    # Check if the image can accommodate the secret text
    image_capacity = image.width * image.height * 3
    if len(binary_secret_text) > image_capacity:
        raise ValueError("Image does not have sufficient capacity to hide the secret text.")
    
    # Hide the secret text in the image
    index = 0
    for i in range(image.width):
        for j in range(image.height):
            r, g, b = pixels[i, j]
            
            # Modify the least significant bit of each color channel
            if index < len(binary_secret_text):
                r = (r & 0xFE) | int(binary_secret_text[index])
                index += 1
            if index < len(binary_secret_text):
                g = (g & 0xFE) | int(binary_secret_text[index])
                index += 1
            if index < len(binary_secret_text):
                b = (b & 0xFE) | int(binary_secret_text[index])
                index += 1
            
            pixels[i, j] = (r, g, b)
            
            if index >= len(binary_secret_text):
                break
        if index >= len(binary_secret_text):
            break
    
    # Save the image with the hidden secret text
    image.save(output_path)
    print("Secret text hidden successfully.")

## 1.2. Decode the message from the image

### Steps for Extracting and Decoding the Message

#### 1. Extracting the Binary Data
The **Least Significant Bits (LSBs)** of the pixel colors are read to reconstruct the binary message.

#### 2. Rebuilding the Message
The binary data is converted back into readable text.

#### 3. Checking the Passcode
The first part of the message contains the hashed passcode. If it matches the hash of the user’s input passcode, the message is considered valid.

#### 4. Getting the Secret Message
Once the passcode is validated, the rest of the message is extracted, and the `"Ending"` marker is removed.


In [138]:
def decode_message_lsb_with_passcode(image_path, passcode):
    # Generate a hashed passcode using SHA-256
    hashed_passcode = hashlib.sha256(passcode.encode('utf-8')).hexdigest()
    
    # Open the image
    image = Image.open(image_path)
    pixels = image.load()
    
    # Extract the secret text from the image
    binary_secret_text = ""
    for i in range(image.width):
        for j in range(image.height):
            r, g, b = pixels[i, j]
            
            # Extract the least significant bit of each color channel
            binary_secret_text += str(r & 1)
            binary_secret_text += str(g & 1)
            binary_secret_text += str(b & 1)
    
    # Convert the binary text to ASCII
    binary_chunks = [binary_secret_text[i:i+8] for i in range(0, len(binary_secret_text), 8)]
    secret_text = ""
    for chunk in binary_chunks:
        char = chr(int(chunk, 2))
        secret_text += char
        if secret_text.endswith("Ending"):
            break
    
    # Remove the delimiter
    extracted_passcode = secret_text[:64]  # Extract the first 64 characters (hashed passcode)
    if extracted_passcode != hashed_passcode:
        raise ValueError("Incorrect passcode.")
    
    return secret_text[64:].replace("Ending", "")  # Remove the passcode and delimiter

### 1.3. Usage of Encoding

In [139]:
encode_message_lsb_with_passcode("pic1.jpg", "hello I am fahad Rafique", "passcode_encoded_image.png", "fsb78")

Secret text hidden successfully.


### 1.4. Usage of Decoding

In [140]:
decoded_message = decode_message_lsb_with_passcode("passcode_encoded_image.png", "fsb78")
print("Decoded message:", decoded_message)

Decoded message: hello I am fahad Rafique
